In [1]:
# ========================== Imports 

import pandas as pd
import numpy as np
import GetPoolData
import itertools
import scipy
from scipy import stats
# Create config.py in this directory and enter your own Bitquery API Token
from config import BITQUERY_API_TOKEN

import ResetStrategy
import ActiveStrategyFramework


c:\Users\newton\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:

#################
#
# Download data
#
################

# Get Swap Data
DOWNLOAD_DATA      = False
address            = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'
token_0_address    = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
token_1_address    = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
flipside_queries   = ['https://api.flipsidecrypto.com/api/v2/queries/b8ad3087-803a-478b-9ed3-c4f3c096bc47/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/de277680-5ff6-4d58-bfff-29ef114215be/data/latest',
                      'https://api.flipsidecrypto.com/api/v2/queries/669fc9f7-8b2d-4dd2-948f-668de9eb4e8a/data/latest']

# Price data for theThese must be month ends
# ["26-4","5-11","12-18","19-25","26-1","2-8","9-15","16-22","23-29","30-6","7-13","16-20","21-27","28-3","4-10","11-17","18-24","25-1"]
price_data_begin         = '2022-01-12'
price_data_end           = '2022-02-18'
file_name                = 'eth_usdc'

swap_data                = GetPoolData.get_pool_data_flipside(address,flipside_queries,file_name,DOWNLOAD_DATA)
price_data               = GetPoolData.get_price_data_bitquery(token_0_address,token_1_address,price_data_begin,price_data_end,BITQUERY_API_TOKEN,file_name,DOWNLOAD_DATA)
# If price data not being downloaded, initialize the flag seperately.

In [ ]:
price_data

In [ ]:
#################
#
# Run Reset Strategy
#
################

# Select date ranges for strategy simulation
# DATE_BEGIN                 = pd.to_datetime(f'2022-06-01 00:00PM',utc=True)
# DATE_END                   = pd.to_datetime(f'2022-06-20 00:00PM',utc=True)
DATE_BEGIN                 = pd.to_datetime(f'{price_data_begin} 00:00PM',utc=True)
DATE_END                   = pd.to_datetime(f'{price_data_end} 00:00PM',utc=True)


# Data for strategy simulation. We can use aggregate_price_data to analyze the strategy at a coarser STRATEGY_FREQUENCY in minutes
z_score_cutoff                           = 5
window_size                              = 60*12
STRATEGY_FREQUENCY                       = '10 min' 
simulate_data_filtered                   = ActiveStrategyFramework.aggregate_price_data(price_data,STRATEGY_FREQUENCY)
simulate_data_filtered_roll              = simulate_data_filtered.quotePrice.rolling(window=window_size) 
simulate_data_filtered['roll_median']    = simulate_data_filtered_roll.median()
roll_dev                                 = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median)
simulate_data_filtered['median_abs_dev'] = 1.4826*roll_dev.rolling(window=window_size).median() # sd(y)/mad(y, const=1) = 1.4826
outlier_indices                          = np.abs(simulate_data_filtered.quotePrice - simulate_data_filtered.roll_median) >= z_score_cutoff*simulate_data_filtered['median_abs_dev']
simulate_data_price                      = simulate_data_filtered[~outlier_indices]['quotePrice'][DATE_BEGIN:DATE_END]

# Data for strategy estimation
STATISTICAL_FREQUENCY        = '8H' 
statistical_data             = ActiveStrategyFramework.aggregate_price_data(price_data,STATISTICAL_FREQUENCY)
statistical_data['z_scores'] = np.abs(scipy.stats.zscore(statistical_data['quotePrice']))
statistical_data             = statistical_data.drop(statistical_data[abs(statistical_data.z_scores) > 3].index)
statistical_data_filtered    = statistical_data.sort_index()

# Initial Position Details
INITIAL_TOKEN_0            = 100000
INITIAL_TOKEN_1            = INITIAL_TOKEN_0*simulate_data_price[0]
FEE_TIER                   = 0.003

# Set decimals according to your pool
DECIMALS_0 = 6
DECIMALS_1 = 18
swap_data['virtual_liquidity'] = swap_data['VIRTUAL_LIQUIDITY_ADJUSTED']*(10**((DECIMALS_1  + DECIMALS_0)/2))
swap_data['traded_in']         = swap_data.apply(lambda x: -x['amount0'] if (x['amount0'] < 0) else -x['amount1'],axis=1).astype(float)
swap_data['traded_out']        = swap_data.apply(lambda x:  x['amount0'] if (x['amount0'] > 0) else  x['amount1'],axis=1).astype(float)

# Strategy Parameters
alpha_range = [.50]
tau_range   = [.50]
# limit_range = [.40]



In [ ]:
pd.DataFrame(simulate_data_filtered)


In [ ]:

sim_performance  = []
sim_results      = []
# for j in list(itertools.product(alpha_range,tau_range)):
# for x in np.linspace(0.1,1,num=19):
# for x in np.linspace(0.5,0.6,num=3):
for x in np.linspace(0.5,0.95,num=10):
    # for y in [round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(x).__round__(2) ]:
    for y in [round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(x).__round__(2) ]:
        print(x,y)
        Reset_Strategy                   = ResetStrategy.ResetStrategy(statistical_data_filtered,x,y)
        simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,swap_data,Reset_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
        sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,Reset_Strategy)
        strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data)
        sim_data.columns
        strat_result['alpha_param']      = x
        strat_result['tau_param']        = y
        # strat_result['limit_param']      =
        sim_results.append(simulated_strategy)
        sim_performance.append(strat_result)
    print("")
    

In [ ]:

sim_performance  = []
sim_results      = []

x, y = 0.75,0.5 

Reset_Strategy                   = ResetStrategy.ResetStrategy(statistical_data_filtered,x,y)
simulated_strategy               = ActiveStrategyFramework.simulate_strategy(simulate_data_price,swap_data,Reset_Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,FEE_TIER,DECIMALS_0,DECIMALS_1)
sim_data                         = ActiveStrategyFramework.generate_simulation_series(simulated_strategy,Reset_Strategy)
strat_result                     = ActiveStrategyFramework.analyze_strategy(sim_data)
sim_data.columns
strat_result['alpha_param']      = x
strat_result['tau_param']        = y
# strat_result['limit_param']      =
sim_results.append(simulated_strategy)
sim_performance.append(strat_result)
    

In [ ]:

pd.DataFrame(sim_performance).to_csv(f"./{price_data_begin}--{price_data_end}-{STRATEGY_FREQUENCY}-{STATISTICAL_FREQUENCY}.csv")

with open(f"./{price_data_begin}--{price_data_end}-{STRATEGY_FREQUENCY}-{STATISTICAL_FREQUENCY}.txt","w") as file:
    file.write(pd.DataFrame(sim_performance).to_string())

In [ ]:
df = pd.DataFrame(sim_performance)
df

In [ ]:
df["net_return"].max()

In [ ]:
df.iloc[df["net_return"].argmax(),:]

In [ ]:
reset = ResetStrategy.ResetStrategy(statistical_data_filtered,0.5,0.4)

base_range_lower      = (1 + reset.inverse_ecdf((1 -      reset.alpha_param)/2))  * simulate_data_price[0]
base_range_upper      = (1 + reset.inverse_ecdf( 1 - (1 - reset.alpha_param)/2))  * simulate_data_price[0]

print(base_range_lower, base_range_upper)

In [ ]:
import math

decimal_adjustment          = 10**(18  - 6)
tickSpacing = 60

# Lower Range
TICK_A_PRE         = int(math.log(decimal_adjustment*base_range_lower,1.0001))
TICK_A             = int(round(TICK_A_PRE/tickSpacing)*tickSpacing)

# Upper Range
TICK_B_PRE        = int(math.log(decimal_adjustment*base_range_upper,1.0001))
TICK_B            = int(round(TICK_B_PRE/tickSpacing)*tickSpacing)


print(TICK_A,TICK_B)

In [ ]:
eth_A = 1 / ((1.0001**TICK_A)/10**18 * 10**6)
eth_B = 1 / ((1.0001**TICK_B)/10**18 * 10**6)

print(eth_A,eth_B)

In [ ]:
import seaborn as sns
# doosra
sns.set_style("darkgrid", {'axes.labelcolor': 'white'})
sns.ecdfplot(x = 'price_return', data = statistical_data_filtered)

In [ ]:
for j in list(itertools.product([1,11,111],[2])):
    print(j[0],j[1])

In [ ]:
for i in np.linspace(0.5,0.95,num=10):
    print(float(i).__round__(2))

In [ ]:
[round(0.05 * i,2)  for i in range(1,20) if i*0.05 < float(0.5).__round__(2) ]

In [ ]:
import pickle
import json



In [ ]:
with open('./data/eth_usdc_swap.pkl', 'rb') as input:
    request_swap = pickle.load(input)

request_swap


with open('./swap.json', 'w') as file:
    file.writelines(json.dumps(request_swap))

In [ ]:
json.dumps(request_swap)